In [1]:
import sys
import os
import glob
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import networkx as nx 
%matplotlib inline

In [54]:
df = pd.read_csv('../../01 Data/03 Consolidated/paper_and_name_lvl_data.csv')

In [55]:
df

,paper_id,name,affiliationIds,new_fname,year,gender_y,paper_count,start_yr,end_yr,age,...,tot_auth_age,tot_M_auth,tot_F_auth,tot_NA_auth,tot_collab,tot_collab_age,tot_collab_pprs,tot_M_collab,tot_F_collab,tot_NA_collab
0,10.1103/PhysRevApplied.9.039901,Tomoyuki Sasaki,[],Tomoyuki,2018,NaN,5,2011,2018,8,...,102,3,0,5,7,94,103,3,0,4
1,10.1103/PhysRevApplied.9.039901,Yuichiro Ando,[],Yuichiro,2018,NaN,10,2012,2018,7,...,102,3,0,5,7,95,98,3,0,4
2,10.1103/PhysRevApplied.9.039901,Makoto Kameno,[],Makoto,2018,male,4,2012,2018,7,...,102,3,0,5,7,95,104,2,0,5
3,10.1103/PhysRevApplied.9.039901,Takayuki Tahara,[],Takayuki,2018,male,3,2014,2018,5,...,102,3,0,5,7,97,105,2,0,5
4,10.1103/PhysRevApplied.9.039901,Hayato Koike,[],Hayato,2018,NaN,4,2014,2018,5,...,102,3,0,5,7,97,104,3,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2196466,10.1103/PhysRevSTAB.14.014001,Mikhail Zobov,a2,Mikhail,2011,male,4,2003,2011,9,...,16,4,0,0,3,7,5,3,0,0
2196467,10.1103/PhysRevSTAB.14.090401,A. Lorusso,a1,A.,2011,NaN,2,2011,2013,3,...,15,0,0,4,3,12,7,0,0,3
2196468,10.1103/PhysRevSTAB.14.090401,F. Gontad,a1,F.,2011,NaN,2,2011,2013,3,...,15,0,0,4,3,12,7,0,0,3
2196469,10.1103/PhysRevSTAB.14.090401,A. Perrone,a1,A.,2011,NaN,4,2006,2013,8,...,15,0,0,4,3,7,5,0,0,3


In [56]:
# df[df.name == 'Tomoyuki Sasaki']

In [57]:
df.columns

Index(['paper_id', 'name', 'affiliationIds', 'new_fname', 'year', 'gender_y',
       'paper_count', 'start_yr', 'end_yr', 'age', 'isMale', 'isFemale',
       'isgenderNA', 'tot_auth', 'tot_auth_papers', 'tot_auth_age',
       'tot_M_auth', 'tot_F_auth', 'tot_NA_auth', 'tot_collab',
       'tot_collab_age', 'tot_collab_pprs', 'tot_M_collab', 'tot_F_collab',
       'tot_NA_collab'],
      dtype='object')

In [58]:
df.drop(columns=['affiliationIds',  'new_fname'], inplace=True, axis=1) 

In [59]:
df.gender_y.fillna('NA', inplace=True)

In [60]:
df

,paper_id,name,year,gender_y,paper_count,start_yr,end_yr,age,isMale,isFemale,...,tot_auth_age,tot_M_auth,tot_F_auth,tot_NA_auth,tot_collab,tot_collab_age,tot_collab_pprs,tot_M_collab,tot_F_collab,tot_NA_collab
0,10.1103/PhysRevApplied.9.039901,Tomoyuki Sasaki,2018,NA,5,2011,2018,8,0,0,...,102,3,0,5,7,94,103,3,0,4
1,10.1103/PhysRevApplied.9.039901,Yuichiro Ando,2018,NA,10,2012,2018,7,0,0,...,102,3,0,5,7,95,98,3,0,4
2,10.1103/PhysRevApplied.9.039901,Makoto Kameno,2018,male,4,2012,2018,7,1,0,...,102,3,0,5,7,95,104,2,0,5
3,10.1103/PhysRevApplied.9.039901,Takayuki Tahara,2018,male,3,2014,2018,5,1,0,...,102,3,0,5,7,97,105,2,0,5
4,10.1103/PhysRevApplied.9.039901,Hayato Koike,2018,NA,4,2014,2018,5,0,0,...,102,3,0,5,7,97,104,3,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2196466,10.1103/PhysRevSTAB.14.014001,Mikhail Zobov,2011,male,4,2003,2011,9,1,0,...,16,4,0,0,3,7,5,3,0,0
2196467,10.1103/PhysRevSTAB.14.090401,A. Lorusso,2011,NA,2,2011,2013,3,0,0,...,15,0,0,4,3,12,7,0,0,3
2196468,10.1103/PhysRevSTAB.14.090401,F. Gontad,2011,NA,2,2011,2013,3,0,0,...,15,0,0,4,3,12,7,0,0,3
2196469,10.1103/PhysRevSTAB.14.090401,A. Perrone,2011,NA,4,2006,2013,8,0,0,...,15,0,0,4,3,7,5,0,0,3


***First Yr Last Yr Columns***

In [61]:
# Author level BOOL val if First Yr or Last Yr
df['First_Yr'] = df['year'] == df['start_yr']
df['Last_Yr'] =  df['year'] == df['end_yr']

In [62]:
df

,paper_id,name,year,gender_y,paper_count,start_yr,end_yr,age,isMale,isFemale,...,tot_F_auth,tot_NA_auth,tot_collab,tot_collab_age,tot_collab_pprs,tot_M_collab,tot_F_collab,tot_NA_collab,First_Yr,Last_Yr
0,10.1103/PhysRevApplied.9.039901,Tomoyuki Sasaki,2018,NA,5,2011,2018,8,0,0,...,0,5,7,94,103,3,0,4,False,True
1,10.1103/PhysRevApplied.9.039901,Yuichiro Ando,2018,NA,10,2012,2018,7,0,0,...,0,5,7,95,98,3,0,4,False,True
2,10.1103/PhysRevApplied.9.039901,Makoto Kameno,2018,male,4,2012,2018,7,1,0,...,0,5,7,95,104,2,0,5,False,True
3,10.1103/PhysRevApplied.9.039901,Takayuki Tahara,2018,male,3,2014,2018,5,1,0,...,0,5,7,97,105,2,0,5,False,True
4,10.1103/PhysRevApplied.9.039901,Hayato Koike,2018,NA,4,2014,2018,5,0,0,...,0,5,7,97,104,3,0,4,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2196466,10.1103/PhysRevSTAB.14.014001,Mikhail Zobov,2011,male,4,2003,2011,9,1,0,...,0,0,3,7,5,3,0,0,False,True
2196467,10.1103/PhysRevSTAB.14.090401,A. Lorusso,2011,NA,2,2011,2013,3,0,0,...,0,4,3,12,7,0,0,3,True,False
2196468,10.1103/PhysRevSTAB.14.090401,F. Gontad,2011,NA,2,2011,2013,3,0,0,...,0,4,3,12,7,0,0,3,True,False
2196469,10.1103/PhysRevSTAB.14.090401,A. Perrone,2011,NA,4,2006,2013,8,0,0,...,0,4,3,7,5,0,0,3,False,False


In [63]:
# df[df.name=='Tomoyuki Sasaki']

In [64]:
# Only keep first and last yerar data
df = df[(df['First_Yr']==True) | (df['Last_Yr']==True)]

In [66]:
# df[df.name=='Tomoyuki Sasaki']

In [67]:
df.columns

Index(['paper_id', 'name', 'year', 'gender_y', 'paper_count', 'start_yr',
       'end_yr', 'age', 'isMale', 'isFemale', 'isgenderNA', 'tot_auth',
       'tot_auth_papers', 'tot_auth_age', 'tot_M_auth', 'tot_F_auth',
       'tot_NA_auth', 'tot_collab', 'tot_collab_age', 'tot_collab_pprs',
       'tot_M_collab', 'tot_F_collab', 'tot_NA_collab', 'First_Yr', 'Last_Yr'],
      dtype='object')

***Rollup Data at Author-Year level for First-Last Paper Analysis***

In [77]:
df_grp = df.groupby(['name', 'year', 'gender_y', 'start_yr','end_yr','age'], as_index=False).agg(
    {'tot_collab':'sum', 'tot_collab_age':'sum', 'tot_collab_pprs':'sum', 
     'tot_M_collab':'sum', 'tot_F_collab':'sum', 'tot_NA_collab':'sum'})

In [78]:
df_grp

,name,year,gender_y,start_yr,end_yr,age,tot_collab,tot_collab_age,tot_collab_pprs,tot_M_collab,tot_F_collab,tot_NA_collab
0,\nDUET Collaboration\n,2017,NA,2017,2017,1,22,420,282,0,0,22
1,\nSπRIT Collaboration\n,2017,NA,2017,2017,1,18,297,544,0,0,18
2,Arianto,2011,NA,2011,2011,1,4,6,6,0,0,4
3,Riazuddin,2011,NA,2011,2011,1,1,29,25,0,0,1
4,A. M. Balbashov,2004,NA,2004,2004,1,7,134,540,0,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...
674720,А. N. Lodygin,2016,NA,2016,2018,3,16,274,230,1,0,15
674721,А. N. Lodygin,2018,NA,2016,2018,3,16,279,240,2,0,14
674722,О. S. Vaulina,2015,NA,2015,2015,1,1,6,2,0,0,1
674723,Ṣ. Uǧur,2015,NA,2015,2015,1,5,94,169,0,0,5


In [79]:
set(df_grp.gender_y)

{'NA', 'female', 'male'}

***For All Ages (Incl. age=1)***

In [114]:
print('Male: Avg Collbortaor Age for FIRST Paper in Lifetime (incl. age=1): {0:0.1f} years'.format(sum(df_grp.tot_collab_age[(df_grp['gender_y'] == 'male') & (df_grp['year'] == df_grp['start_yr'])])/sum(df_grp.tot_collab[(df_grp['gender_y'] == 'male') & (df_grp['year'] == df_grp['start_yr'])])))
print('Male: Avg Collbortaor Age for LAST Paper in Lifetime (incl. age=1): {0:0.1f} years'.format(sum(df_grp.tot_collab_age[(df_grp['gender_y'] == 'male') & (df_grp['year'] == df_grp['end_yr'])])/sum(df_grp.tot_collab[(df_grp['gender_y'] == 'male') & (df_grp['year'] == df_grp['end_yr'])])))

Male: Avg Collbortaor Age for FIRST Paper in Lifetime (incl. age=1): 12.5 years
Male: Avg Collbortaor Age for LAST Paper in Lifetime (incl. age=1): 11.1 years


In [118]:
print('Female: Avg Collbortaor Age for FIRST Paper in Lifetime (incl. age=1): {0:0.1f} years'.format(sum(df_grp.tot_collab_age[(df_grp['gender_y'] == 'female') & (df_grp['year'] == df_grp['start_yr'])])/sum(df_grp.tot_collab[(df_grp['gender_y'] == 'female') & (df_grp['year'] == df_grp['start_yr'])])))
print('Female: Avg Collbortaor Age for LAST Paper in Lifetime (incl. age=1): {0:0.1f} years'.format(sum(df_grp.tot_collab_age[(df_grp['gender_y'] == 'female') & (df_grp['year'] == df_grp['end_yr'])])/sum(df_grp.tot_collab[(df_grp['gender_y'] == 'female') & (df_grp['year'] == df_grp['end_yr'])])))

Female: Avg Collbortaor Age for FIRST Paper in Lifetime (incl. age=1): 11.8 years
Female: Avg Collbortaor Age for LAST Paper in Lifetime (incl. age=1): 10.9 years


***For Age > 1***

In [143]:
print('Male: Avg Collbortaor Age for FIRST Paper in Lifetime (age > 1yr): {0:0.0f} years'.format(sum(df_grp.tot_collab_age[(df_grp['gender_y'] == 'male') & (df_grp['year'] == df_grp['start_yr']) & (df_grp['age'] > 1)])/sum(df_grp.tot_collab[(df_grp['gender_y'] == 'male') & (df_grp['year'] == df_grp['start_yr']) & (df_grp['age'] > 1)])))
print('Male: Avg Collbortaor Age for LAST Paper in Lifetime (age > 1yr): {0:0.0f} years'.format(sum(df_grp.tot_collab_age[(df_grp['gender_y'] == 'male') & (df_grp['year'] == df_grp['end_yr']) & (df_grp['age'] > 1)])/sum(df_grp.tot_collab[(df_grp['gender_y'] == 'male') & (df_grp['year'] == df_grp['end_yr'])& (df_grp['age'] > 1)])))

Male: Avg Collbortaor Age for FIRST Paper in Lifetime (age > 1yr): 15 years
Male: Avg Collbortaor Age for LAST Paper in Lifetime (age > 1yr): 11 years


In [142]:
print('Female: Avg Collbortaor Age for FIRST Paper in Lifetime (age > 1yr): {0:0.0f} years'.format(sum(df_grp.tot_collab_age[(df_grp['gender_y'] == 'female') & (df_grp['year'] == df_grp['start_yr']) & (df_grp['age'] > 1)])/sum(df_grp.tot_collab[(df_grp['gender_y'] == 'female') & (df_grp['year'] == df_grp['start_yr']) & (df_grp['age'] > 1)])))
print('Female: Avg Collbortaor Age for LAST Paper in Lifetime (age > 1yr): {0:0.0f} years'.format(sum(df_grp.tot_collab_age[(df_grp['gender_y'] == 'female') & (df_grp['year'] == df_grp['end_yr']) & (df_grp['age'] > 1)])/sum(df_grp.tot_collab[(df_grp['gender_y'] == 'female') & (df_grp['year'] == df_grp['end_yr'])& (df_grp['age'] > 1)])))

Female: Avg Collbortaor Age for FIRST Paper in Lifetime (age > 1yr): 14 years
Female: Avg Collbortaor Age for LAST Paper in Lifetime (age > 1yr): 11 years


***Collab Paper Count***

In [144]:
print('Male: Avg Collaborator #Papers for FIRST Paper in Lifetime (age > 1yr): {0:0.1f} papers'.format(sum(df_grp.tot_collab_pprs[(df_grp['gender_y'] == 'male') & (df_grp['year'] == df_grp['start_yr']) & (df_grp['age'] > 1)])/sum(df_grp.tot_collab[(df_grp['gender_y'] == 'male') & (df_grp['year'] == df_grp['start_yr']) & (df_grp['age'] > 1)])))
print('Male: Avg Collaborator #Papers for LAST Paper in Lifetime (age > 1yr): {0:0.1f} papers'.format(sum(df_grp.tot_collab_pprs[(df_grp['gender_y'] == 'male') & (df_grp['year'] == df_grp['end_yr']) & (df_grp['age'] > 1)])/sum(df_grp.tot_collab[(df_grp['gender_y'] == 'male') & (df_grp['year'] == df_grp['end_yr'])& (df_grp['age'] > 1)])))

Male: Avg Collaborator #Papers for FIRST Paper in Lifetime (age > 1yr): 20.9 papers
Male: Avg Collaborator #Papers for LAST Paper in Lifetime (age > 1yr): 16.1 papers


In [151]:
print('Female: Avg Collaborator #Papers for FIRST Paper in Lifetime (age > 1yr): {0:0.0f} papers'.format(sum(df_grp.tot_collab_pprs[(df_grp['gender_y'] == 'female') & (df_grp['year'] == df_grp['start_yr']) & (df_grp['age'] > 1)])/sum(df_grp.tot_collab[(df_grp['gender_y'] == 'female') & (df_grp['year'] == df_grp['start_yr']) & (df_grp['age'] > 1)])))
print('Female: Avg Collaborator #Papers for LAST Paper in Lifetime (age > 1yr): {0:0.0f} papers'.format(sum(df_grp.tot_collab_pprs[(df_grp['gender_y'] == 'female') & (df_grp['year'] == df_grp['end_yr']) & (df_grp['age'] > 1)])/sum(df_grp.tot_collab[(df_grp['gender_y'] == 'female') & (df_grp['year'] == df_grp['end_yr'])& (df_grp['age'] > 1)])))

Female: Avg Collaborator #Papers for FIRST Paper in Lifetime (age > 1yr): 21 papers
Female: Avg Collaborator #Papers for LAST Paper in Lifetime (age > 1yr): 17 papers


In [141]:
print('Male: Avg Collaborator #Papers for FIRST Paper in Lifetime (incl. age=1): {0:0.0f} papers'.format(sum(df_grp.tot_collab_pprs[(df_grp['gender_y'] == 'male') & (df_grp['year'] == df_grp['start_yr'])])/sum(df_grp.tot_collab[(df_grp['gender_y'] == 'male') & (df_grp['year'] == df_grp['start_yr'])])))
print('Male: Avg Collaborator #Papers for LAST Paper in Lifetime (incl. age=1): {0:0.0f} papers'.format(sum(df_grp.tot_collab_pprs[(df_grp['gender_y'] == 'male') & (df_grp['year'] == df_grp['end_yr'])])/sum(df_grp.tot_collab[(df_grp['gender_y'] == 'male') & (df_grp['year'] == df_grp['end_yr'])])))
print('\n')
print('Female: Avg Collaborator #Papers for FIRST Paper in Lifetime (incl. age=1): {0:0.0f} papers'.format(sum(df_grp.tot_collab_pprs[(df_grp['gender_y'] == 'female') & (df_grp['year'] == df_grp['start_yr'])])/sum(df_grp.tot_collab[(df_grp['gender_y'] == 'female') & (df_grp['year'] == df_grp['start_yr'])])))
print('Female: Avg Collaborator #Papers for LAST Paper in Lifetime (incl. age=1): {0:0.0f} papers'.format(sum(df_grp.tot_collab_pprs[(df_grp['gender_y'] == 'female') & (df_grp['year'] == df_grp['end_yr'])])/sum(df_grp.tot_collab[(df_grp['gender_y'] == 'female') & (df_grp['year'] == df_grp['end_yr'])])))

Male: Avg Collaborator #Papers for FIRST Paper in Lifetime (incl. age=1): 16 papers
Male: Avg Collaborator #Papers for LAST Paper in Lifetime (incl. age=1): 14 papers


Female: Avg Collaborator #Papers for FIRST Paper in Lifetime (incl. age=1): 16 papers
Female: Avg Collaborator #Papers for LAST Paper in Lifetime (incl. age=1): 15 papers


***Avg Collab Count***

In [137]:
print('Male: Avg #Collaborator for FIRST Paper in Lifetime (incl. age=1): {0:0.0f} authors'.format(np.mean(df_grp.tot_collab[(df_grp['gender_y'] == 'male') & (df_grp['year'] == df_grp['start_yr'])])))
print('Male: Avg #Collaborator for LAST Paper in Lifetime (incl. age=1): {0:0.0f} authors'.format(np.mean(df_grp.tot_collab[(df_grp['gender_y'] == 'male') & (df_grp['year'] == df_grp['end_yr'])])))
print('\n')
print('Female: Avg #Collaborator for FIRST Paper in Lifetime (incl. age=1): {0:0.0f} authors'.format(np.mean(df_grp.tot_collab[(df_grp['gender_y'] == 'female') & (df_grp['year'] == df_grp['start_yr'])])))
print('Female: Avg #Collaborator for LAST Paper in Lifetime (incl. age=1): {0:0.0f} authors'.format(np.mean(df_grp.tot_collab[(df_grp['gender_y'] == 'female') & (df_grp['year'] == df_grp['end_yr'])])))

Male: Avg #Collaborator for FIRST Paper in Lifetime (incl. age=1): 4 authors
Male: Avg #Collaborator for LAST Paper in Lifetime (incl. age=1): 4 authors


Female: Avg #Collaborator for FIRST Paper in Lifetime (incl. age=1): 4 authors
Female: Avg #Collaborator for LAST Paper in Lifetime (incl. age=1): 5 authors


In [138]:
print('Male: Avg #Collaborator for FIRST Paper in Lifetime (age > 1yr): {0:0.0f} authors'.format(np.mean(df_grp.tot_collab[(df_grp['gender_y'] == 'male') & (df_grp['year'] == df_grp['start_yr'])& (df_grp['age'] > 1)])))
print('Male: Avg #Collaborator for LAST Paper in Lifetime (age > 1yr): {0:0.0f} authors'.format(np.mean(df_grp.tot_collab[(df_grp['gender_y'] == 'male') & (df_grp['year'] == df_grp['end_yr'])& (df_grp['age'] > 1)])))
print('\n')
print('Female: Avg #Collaborator for FIRST Paper in Lifetime (age > 1yr): {0:0.0f} authors'.format(np.mean(df_grp.tot_collab[(df_grp['gender_y'] == 'female') & (df_grp['year'] == df_grp['start_yr'])& (df_grp['age'] > 1)])))
print('Female: Avg #Collaborator for LAST Paper in Lifetime (age > 1yr): {0:0.0f} authors'.format(np.mean(df_grp.tot_collab[(df_grp['gender_y'] == 'female') & (df_grp['year'] == df_grp['end_yr'])& (df_grp['age'] > 1)])))

Male: Avg #Collaborator for FIRST Paper in Lifetime (age > 1yr): 3 authors
Male: Avg #Collaborator for LAST Paper in Lifetime (age > 1yr): 5 authors


Female: Avg #Collaborator for FIRST Paper in Lifetime (age > 1yr): 4 authors
Female: Avg #Collaborator for LAST Paper in Lifetime (age > 1yr): 5 authors


***Male/Female Ratio***

In [1]:
# print('Male: Avg # Male Collaborator for FIRST Paper (age > 1yr): {0:0.0f} authors'.format(np.mean(df_grp.tot_M_collab[(df_grp['gender_y'] == 'male') & (df_grp['year'] == df_grp['start_yr'])& (df_grp['age'] > 1)])))
# print('Male: Avg # Male Collaborator for LAST Paper (age > 1yr): {0:0.0f} authors'.format(np.mean(df_grp.tot_M_collab[(df_grp['gender_y'] == 'male') & (df_grp['year'] == df_grp['end_yr'])& (df_grp['age'] > 1)])))
# print('\n')
# print('Female: Avg # Male Collaborator for FIRST Paper (age > 1yr): {0:0.0f} authors'.format(np.mean(df_grp.tot_M_collab[(df_grp['gender_y'] == 'female') & (df_grp['year'] == df_grp['start_yr'])& (df_grp['age'] > 1)])))
# print('Female: Avg # Male Collaborator for LAST Paper (age > 1yr): {0:0.0f} authors'.format(np.mean(df_grp.tot_M_collab[(df_grp['gender_y'] == 'female') & (df_grp['year'] == df_grp['end_yr'])& (df_grp['age'] > 1)])))

In [150]:
# print('Male: Avg # Female Collaborator for FIRST Paper (age > 1yr): {0:0.0f} authors'.format(np.mean(df_grp.tot_F_collab[(df_grp['gender_y'] == 'male') & (df_grp['year'] == df_grp['start_yr'])& (df_grp['age'] > 1)])))
# print('Male: Avg # Female Collaborator for LAST Paper (age > 1yr): {0:0.0f} authors'.format(np.mean(df_grp.tot_F_collab[(df_grp['gender_y'] == 'male') & (df_grp['year'] == df_grp['end_yr'])& (df_grp['age'] > 1)])))
# print('\n')
# print('Female: Avg # Female Collaborator for FIRST Paper (age > 1yr): {0:0.0f} authors'.format(np.mean(df_grp.tot_F_collab[(df_grp['gender_y'] == 'female') & (df_grp['year'] == df_grp['start_yr'])& (df_grp['age'] > 1)])))
# print('Female: Avg # Female Collaborator for LAST Paper (age > 1yr): {0:0.0f} authors'.format(np.mean(df_grp.tot_F_collab[(df_grp['gender_y'] == 'female') & (df_grp['year'] == df_grp['end_yr'])& (df_grp['age'] > 1)])))